In [2]:
## Here we will write out the matrix elements for our non-eq setup

from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [3]:
## Firstly, we have to define the integral function C and D

def integral1(i,k,tb,beta,mu,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,tb,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + 0.5*(func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))

    return val

def D(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + integral2(i,k,tb,gamma,eigenergies) + 0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))
    return val

In [4]:
NL1 = 2
NL2 = 2
NM = 2

N = NL1 + NL2 + NM
dL1 = 2**NL1
dL2 = 2**NL2
dM = 2**NM
d = 2**N
dims = [2]*N

In [5]:
## Define the Hamiltonian 

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL1)]
create_sm_list_right = [create_sm(N,NM + NL1 + i + 1) for i in range(NL2)]  # Create list of sigma_- operators

eigenergies,eigstates=H_S.eigenstates()

In [6]:
## Let us compute the thermal density matrix
beta1 = 1  #left baths
beta2 = 0.5 #right baths

rho_th = (-beta1*H_S).expm()/((-beta1*H_S).expm()).tr() #left thermal density matrix
print(rho_th)

Quantum object: dims=[[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape=(64, 64), type='oper', dtype=Dense, isherm=True
Qobj data =
[[3.80777093e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.03175783e-03 3.29634248e-06 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.29634248e-06 1.02846676e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.61523826e-02
  1.79974202e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.79974202e-04
  5.63320690e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.53616443e-01]]


In [7]:
## Now we set parameters for the  non-eq setup

beta_list_left = np.linspace(beta1,beta1,NL1)
beta_list_right = np.linspace(beta2,beta2,NL2)

mu_list_left = np.linspace(-0.5,-0.5,NL1)
mu_list_right = np.linspace(-0.5,-0.5,NL2)

gamma_list_left = np.linspace(1,1,NL1)  #coupling strengths to left baths
gamma_list_right = np.linspace(1,1,NL2)  #coupling strengths to right baths

tb = 0.01

In [8]:
number = len(eigenergies)

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(NL1):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_left[l]*proj_y,create_sm_list_left[l].dag())*C(k,y,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_left[l].dag(),proj_y*create_sm_list_left[l]*proj_k)*D(y,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

            for l in range(NL2):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_right[l]*proj_y,create_sm_list_right[l].dag())*C(k,y,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_right[l].dag(),proj_y*create_sm_list_right[l]*proj_k)*D(y,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum


In [9]:
print(number)  #should be 2^N

64


In [10]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]
print(rho_th_diag)


[np.complex128(0.1536164432357538+0j), np.complex128(0.05651233129231378+0j), np.complex128(0.056463896272977196+0j), np.complex128(0.05633178086692867+0j), np.complex128(0.05615180727947186+0j), np.complex128(0.055972408687012254+0j), np.complex128(0.055841443273902114+0j), np.complex128(0.020789724855111807+0j), np.complex128(0.020771906607265127+0j), np.complex128(0.02075165768501958+0j), np.complex128(0.020723304065517863+0j), np.complex128(0.020710947145894693+0j), np.complex128(0.020674601227246826+0j), np.complex128(0.02065709548273863+0j), np.complex128(0.020629746935722004+0j), np.complex128(0.020591098428797645+0j), np.complex128(0.020560107729941855+0j), np.complex128(0.020542918945809906+0j), np.complex128(0.020509188788367952+0j), np.complex128(0.020490634791516974+0j), np.complex128(0.02043729694990147+0j), np.complex128(0.020361504768194885+0j), np.complex128(0.007648112361806576+0j), np.complex128(0.007641557394746085+0j), np.complex128(0.007634108232546068+0j), np.comp

In [11]:
## Now we use the lin algebra solver to solve the equation Ax = b, where b is all zeroes

b = np.zeros((number),dtype=complex)

x = np.linalg.solve(A,b)

print(x)

[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.-0.j
 -0.-0.j]


In [12]:
print(A)

[[ 2.34082148e+000+0.00000000e+000j -1.05026864e+000+0.00000000e+000j
  -1.50581891e+000+0.00000000e+000j ...  0.00000000e+000+0.00000000e+000j
  -2.31309218e-125+0.00000000e+000j  0.00000000e+000+0.00000000e+000j]
 [-3.90235419e-001-6.49843677e-018j  3.00105171e+000+2.61682359e-017j
   9.77188114e-019+1.65765390e-017j ... -1.71032420e-034+5.28869931e-051j
  -7.75229684e-035-2.26855059e-050j -1.11210758e-065-1.34086745e-081j]
 [-5.59188164e-001+2.79859686e-018j  3.23565678e-017-1.37151575e-017j
   3.28765063e+000+5.10343902e-017j ... -9.79756135e-035+1.15715720e-050j
  -4.28643131e-035-1.26575307e-051j  3.81455601e-067+7.72195917e-082j]
 ...
 [ 0.00000000e+000+0.00000000e+000j -4.01315383e-036-1.49470652e-054j
  -1.13612866e-035+7.38571894e-052j ...  5.47225092e+000-4.67291136e-017j
   6.21194047e-018-4.13150151e-017j -1.30529295e+000+7.32920669e-017j]
 [ 3.48028785e-098+7.47416528e-114j -1.81899287e-036-2.85404455e-054j
  -5.07036151e-036-3.38418710e-052j ...  2.80788361e-018-2.458089

In [13]:
## Now we can add one more constraint, that the sum of all the elements of x should be 1, which is basically another row in the matrix A consisting of all 1

#A = np.vstack([A,np.ones((1,number))])

#print(A)

print(np.linalg.matrix_rank(A))

q,r = np.linalg.qr(A.T)

print(r)
diag_r = [r[i][i] for i in range(number)]

print(np.linalg.matrix_rank(A))





63
[[-3.56192881e+00+0.00000000e+00j  1.14134284e+00+4.18663613e-18j
   1.75735297e+00-1.42059621e-17j ... -4.34283045e-34+3.34142653e-51j
  -1.96924663e-34+3.08841995e-51j -6.34002230e-66+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j -3.69232386e+00+0.00000000e+00j
   1.54490042e-01-3.15573685e-17j ... -7.11705698e-32+2.34773060e-48j
  -5.42865828e-32+1.51863357e-48j -1.80305111e-34+1.17170177e-50j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -3.58374196e+00+0.00000000e+00j ... -6.44349201e-32+1.06343132e-48j
  -3.88959613e-32-1.99244231e-48j -1.13743847e-34+2.27617320e-50j]
 ...
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ... -4.70907726e+00+0.00000000e+00j
   1.60082892e+00-1.05909216e-16j  3.10824834e+00+6.79810487e-17j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
  -3.60859910e+00+0.00000000e+00j  3.6085991

In [14]:
print(diag_r)

[np.complex128(-3.5619288054017195+0j), np.complex128(-3.6923238583970783+0j), np.complex128(-3.5837419571612577+0j), np.complex128(-3.4034180287810964+0j), np.complex128(-3.318265983767635+0j), np.complex128(-3.3082773251782305+0j), np.complex128(-3.5430341982550164+0j), np.complex128(-3.785748132109142+0j), np.complex128(-3.808823396943822+0j), np.complex128(-3.4143230975283783+0j), np.complex128(-3.75085938464644+0j), np.complex128(-3.6220128369686786+0j), np.complex128(-3.5463522969718144+0j), np.complex128(-3.730223259819991+0j), np.complex128(-3.6444572504048978+0j), np.complex128(-3.733745524905896+0j), np.complex128(-3.6307586846325806+0j), np.complex128(-3.7904384582863004+0j), np.complex128(-3.5342978743750315+0j), np.complex128(-3.4623054892102822+0j), np.complex128(-3.5413620179075+0j), np.complex128(-3.557334188734087+0j), np.complex128(-3.851709981470998+0j), np.complex128(-4.174031963764874+0j), np.complex128(-4.131725564147+0j), np.complex128(-4.082200446673106+0j), np.

In [15]:
## Since last element of r is 0, we can remove the last row of  A and add a row of 1's

A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])

print(np.linalg.matrix_rank(A_new))
print(A_new.shape)
print(A_new)

64
(64, 64)
[[ 2.34082148e+000+0.00000000e+000j -1.05026864e+000+0.00000000e+000j
  -1.50581891e+000+0.00000000e+000j ...  0.00000000e+000+0.00000000e+000j
  -2.31309218e-125+0.00000000e+000j  0.00000000e+000+0.00000000e+000j]
 [-3.90235419e-001-6.49843677e-018j  3.00105171e+000+2.61682359e-017j
   9.77188114e-019+1.65765390e-017j ... -1.71032420e-034+5.28869931e-051j
  -7.75229684e-035-2.26855059e-050j -1.11210758e-065-1.34086745e-081j]
 [-5.59188164e-001+2.79859686e-018j  3.23565678e-017-1.37151575e-017j
   3.28765063e+000+5.10343902e-017j ... -9.79756135e-035+1.15715720e-050j
  -4.28643131e-035-1.26575307e-051j  3.81455601e-067+7.72195917e-082j]
 ...
 [ 0.00000000e+000+0.00000000e+000j -4.01315383e-036-1.49470652e-054j
  -1.13612866e-035+7.38571894e-052j ...  5.47225092e+000-4.67291136e-017j
   6.21194047e-018-4.13150151e-017j -1.30529295e+000+7.32920669e-017j]
 [ 3.48028785e-098+7.47416528e-114j -1.81899287e-036-2.85404455e-054j
  -5.07036151e-036-3.38418710e-052j ...  2.80788361e-

In [16]:
print(A.shape)

(64, 64)


In [17]:
print(np.linalg.matrix_rank(A))
print(A.shape)

63
(64, 64)


In [18]:
b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)
print(x)

[0.15083039+2.87379435e-18j 0.05604221-1.28877942e-18j
 0.05601111+2.65646298e-18j 0.05592624+2.13414591e-18j
 0.05581052-8.37289613e-20j 0.05569504+2.57615703e-18j
 0.05561065-1.52899617e-18j 0.02082292-1.54340385e-19j
 0.02081137-1.34975904e-20j 0.02079823+1.12194878e-18j
 0.02077983+2.52428368e-19j 0.02077181-3.42086761e-19j
 0.02074821-5.68784778e-19j 0.02073683+6.48437641e-19j
 0.02071906+3.48379069e-19j 0.02069392-8.86336812e-19j
 0.02067376-2.85891187e-19j 0.02066257-3.04879766e-20j
 0.0206406 -5.22273452e-19j 0.02062851+5.05557824e-19j
 0.02059374+6.49889909e-19j 0.02054427-5.09842458e-19j
 0.00773692-1.34499840e-19j 0.00773263-9.72608610e-20j
 0.00772775+1.03451869e-19j 0.00772203-1.31004114e-19j
 0.00772091+1.60845599e-19j 0.00771793-4.23736379e-20j
 0.00770916+1.01120645e-19j 0.00770493+2.06693728e-19j
 0.00769833-4.82454558e-19j 0.00769357-3.82841123e-20j
 0.00768899+1.47340431e-19j 0.0076815 -3.97286643e-19j
 0.00767734+1.40687346e-19j 0.00767308-2.45033084e-19j
 0.0076691

In [19]:
##Let us check if this is correct

print(np.dot(A_new,x))
print(np.dot(A[-1],x))

[ 2.77555756e-17+1.54074396e-33j -5.63747680e-36+0.00000000e+00j
 -5.55111512e-17-2.31111593e-33j  1.38777878e-17+0.00000000e+00j
  1.38777878e-17-7.70371978e-34j -1.68072642e-34+0.00000000e+00j
  7.64203105e-35+7.70371978e-34j  6.93889390e-18-1.44444746e-34j
  6.93889390e-18+7.70371978e-34j -3.60822483e-16+7.70371978e-34j
  7.54978670e-35+7.22223729e-34j -1.38777878e-17+2.31111593e-33j
  6.93889390e-18-1.54074396e-33j  1.38777878e-17-1.15555797e-33j
  1.38777878e-17+1.82963345e-33j -3.46944695e-17+0.00000000e+00j
  6.93889390e-17-1.92592994e-33j  7.93814167e-35-2.88889492e-34j
  4.51028104e-17-2.88889492e-34j  5.55111512e-17+3.85185989e-34j
  6.93889390e-18+7.70371978e-34j  4.85722573e-17+5.77778983e-34j
  2.25397970e-35+1.10740972e-33j  1.21430643e-17-1.92592994e-34j
 -3.03104108e-35+0.00000000e+00j -1.04083409e-17+2.88889492e-34j
  1.38777878e-17-3.85185989e-34j -6.93889390e-18+2.40741243e-34j
 -1.73472348e-18-3.85185989e-34j  8.67361738e-18+0.00000000e+00j
 -3.46944695e-18+3.851859

In [20]:
#We have to tranfer the basis of the solution rho matrix to computational basis from the eigenbasis

x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

#set U matrix whose columns are the eigenvectors of the Hamiltonian

U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

check = 1

print(U[:,check])
print(eigstates[check])

[ 0.00000000e+00+0.j -5.75754671e-84-0.j -7.20269094e-81-0.j
  9.97707536e-26+0.j  0.00000000e+00+0.j -9.97750790e-26+0.j
 -4.68927578e-18+0.j  7.44164867e-30+0.j  6.93446618e-39+0.j
  4.68927558e-18+0.j  2.00350271e-22+0.j  1.62645715e-28+0.j
 -2.76943731e-22+0.j  1.69750167e-16+0.j -1.69218316e-16+0.j
 -3.47076771e-19+0.j  2.16692542e-36+0.j  4.68887508e-18+0.j
  3.61763387e-22+0.j -1.69750168e-16+0.j -2.45816511e-22+0.j
  1.45324297e-17+0.j  7.48129298e-17+0.j  1.36203428e-17+0.j
  4.68870187e-18+0.j -1.46662039e-16+0.j  3.47937418e-17+0.j
  3.84671301e-17+0.j -1.38911464e-16+0.j -1.73142083e-16+0.j
 -1.15759670e-16+0.j -4.08248290e-01+0.j  1.00408924e-33+0.j
 -3.30354054e-22+0.j -4.68835463e-18+0.j  1.40153459e-16+0.j
 -4.68902623e-18+0.j  5.20265203e-17+0.j  2.06555726e-17+0.j
 -1.76440484e-16+0.j -1.19785237e-22+0.j -4.79489729e-17+0.j
  6.25997904e-17+0.j  1.98793246e-16+0.j  2.21017512e-16+0.j
  2.22708655e-16+0.j -2.92101574e-16+0.j -4.08248290e-01+0.j
  2.00903823e-22+0.j -9.

In [21]:
rho_comp = np.dot(np.conjugate(U.T),np.dot(rho,U))

print(rho_comp)

[[ 3.96869452e-04+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  2.76472669e-03+0.j -1.91919191e-03+0.j ...
   1.23382369e-35+0.j  6.14854028e-36+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.91919191e-03+0.j  3.46396810e-03+0.j ...
   2.19576880e-35+0.j  1.09670227e-35+0.j  0.00000000e+00+0.j]
 ...
 [ 0.00000000e+00+0.j  1.23382369e-35+0.j  2.19576880e-35+0.j ...
   3.72577112e-02+0.j -1.60493125e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  6.14854028e-36+0.j  1.09670227e-35+0.j ...
  -1.60493125e-02+0.j  3.80336012e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  1.50830392e-01+0.j]]


In [22]:
def L2_red(rho,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(NL1):
                op = commutator(rho*proj_i*create_sm_list_left[l]*proj_k,create_sm_list_left[l].dag())*C(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies) + commutator(create_sm_list_left[l].dag(),proj_i*create_sm_list_left[l]*proj_k*rho)*D(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += op
                sum += op.dag()

            for l in range(NL2):
                op = commutator(rho*proj_i*create_sm_list_right[l]*proj_k,create_sm_list_right[l].dag())*C(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies) + commutator(create_sm_list_right[l].dag(),proj_i*create_sm_list_right[l]*proj_k*rho)*D(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += op
                sum += op.dag()
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

In [23]:
L2_redfield = L2_red(rho_comp,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb)

In [24]:
print(L2_redfield)

[[-7.27012285e-03+0.00000000e+00j  7.07643897e-05-6.37136687e-06j
   1.13721041e-03+5.65359500e-06j ...  1.00668787e-25+3.18963083e-25j
   1.76352681e-25-2.22026578e-26j  0.00000000e+00+0.00000000e+00j]
 [ 7.07643897e-05+6.37136687e-06j  6.77671500e-03+0.00000000e+00j
  -9.81197069e-03+2.63936399e-06j ... -7.86704063e-21-7.99715198e-21j
  -1.72726837e-21-1.74064237e-21j -3.10789640e-30-1.73843348e-30j]
 [ 1.13721041e-03-5.65359500e-06j -9.81197069e-03-2.63936399e-06j
   1.04669989e-02+0.00000000e+00j ...  6.19233363e-21+6.91143607e-21j
   1.35153944e-21+1.51667208e-21j -1.38814447e-28-4.41998271e-30j]
 ...
 [ 1.00668787e-25-3.18963083e-25j -7.86704063e-21+7.99715198e-21j
   6.19233363e-21-6.91143607e-21j ... -1.13099905e-01+0.00000000e+00j
  -3.52215065e-02-2.97185715e-05j -9.97623448e-04+1.12085895e-05j]
 [ 1.76352681e-25+2.22026578e-26j -1.72726837e-21+1.74064237e-21j
   1.35153944e-21-1.51667208e-21j ... -3.52215065e-02+2.97185715e-05j
  -1.22543918e-01+0.00000000e+00j  3.19535617e-

In [25]:
## As we can see, all the checks are completely satisfied. Thus we can arrange the values of x (real values) in a diagonal density matrix and save it in a .mat file
#We will also pass in L2_redfield operator

rho_th_check = np.zeros((number,number),dtype=complex)
for i in range(number):
    for k in range(number):
        rho_th_check[i,k] = rho_th[i,k]

print(rho_th_check)

data_dict = {"dm_ness":rho_comp, "dm_th":rho_th_new,"L2_red":L2_redfield, "dm_th_check":rho_th_check}

scipy.io.savemat(F'../matlab/ness_data_NL1 = {NL1}_NL2 = {NL2}_NM = {NM}_1.mat',data_dict)


[[3.80777093e-04+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 1.03175783e-03+0.j 3.29634248e-06+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 3.29634248e-06+0.j 1.02846676e-03+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 ...
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  5.61523826e-02+0.j 1.79974202e-04+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  1.79974202e-04+0.j 5.63320690e-02+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 1.53616443e-01+0.j]]
